<a href="https://colab.research.google.com/github/Aru4444/CNN/blob/main/NaturalDisasterJU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clean installation without line continuation comments
!pip install -q \
    transformers==4.51.3 \
    datasets==3.5.1 \
    accelerate==0.29.3 \
    peft==0.15.2 \
    bitsandbytes==0.45.5 \
    sentencepiece==0.2.0 \
    einops==0.7.0 \
    pandas==2.2.2 \
    numpy==2.0.0 \
    fsspec==2025.3.0

# Now you can add comments separately if needed
print("Installed: pandas 2.2.2 (for Colab compatibility)")
print("Installed: numpy 2.0.0 (for thinc compatibility)")

Installed: pandas 2.2.2 (for Colab compatibility)
Installed: numpy 2.0.0 (for thinc compatibility)


In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

# Load your dataset
dataset = load_dataset('json', data_files={'train': '/content/drive/MyDrive/Colab_Notebooks/train_data.json'})

# Split into train/test
split_dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)
print(f"Train samples: {len(split_dataset['train'])}, Test samples: {len(split_dataset['test'])}")


#from datasets import Dataset
#import json
#
## 1. Load with manual cleaning
#with open('/content/drive/MyDrive/Colab_Notebooks/train_data.json') as f:
#    try:
#        data = json.load(f)
#    except json.JSONDecodeError:
#        # For line-delimited JSON
#        data = [json.loads(line) for line in f]
#
## 2. Standardize format
#clean_data = []
#for item in data:
#    try:
#        # Ensure answers is always {'text': [...]}
#        if 'answers' in item:
#            if isinstance(item['answers'], str):
#                item['answers'] = {'text': [item['answers']]}
#            elif isinstance(item['answers'], list):
#                item['answers'] = {'text': item['answers']}
#        clean_data.append(item)
#    except Exception as e:
#        print(f"Skipping malformed item: {e}")
#
## 3. Create dataset
#dataset = Dataset.from_list(clean_data)
#split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
#print(f"Loaded {len(dataset)} examples")
#

Train samples: 1073, Test samples: 269


In [ ]:
!find "/content/drive/MyDrive/" -name "*.json"

/content/drive/MyDrive/Colab_Notebooks/train_data.json


In [ ]:
def format_qwen_qa(example):
    """Handles multiple dataset formats for Qwen fine-tuning"""
    try:
        # Case 1: Standard QA format (context/question/answer)
        if 'answer' in example:
            answer = example['answer']
            context = example['context']
            question = example['question']

        # Case 2: SQUAD-like format (with 'answers' dictionary)
        elif 'answers' in example:
            answers = example['answers']
            if isinstance(answers, dict):
                answer = answers['text'][0] if isinstance(answers['text'], list) else answers['text']
            else:
                answer = answers[0] if isinstance(answers, list) else answers
            context = example['context']
            question = example['question']

        # Case 3: Nested QA format (with 'qas' list)
        elif 'qas' in example:
            qa = example['qas'][0]  # Taking first QA pair if multiple exist
            answers = qa['answers']
            answer = answers['text'][0] if isinstance(answers['text'], list) else answers['text']
            context = example['context']
            question = qa['question']

        else:
            raise ValueError("Unsupported dataset format")

        return {
            "text": f"""<|im_start|>system
You are a natural disaster expert.<|im_end|>
<|im_start|>context
{context}<|im_end|>
<|im_start|>question
{question}<|im_end|>
<|im_start|>answer
{answer}<|im_end|>"""
        }

    except Exception as e:
        print(f"Skipping malformed example: {e}\nExample: {example}")
        return None

# Apply with error filtering
train_dataset = split_dataset['train'].map(format_qwen_qa).filter(lambda x: x is not None)
test_dataset = split_dataset['test'].map(format_qwen_qa).filter(lambda x: x is not None)

# Verify
print(f"Kept {len(train_dataset)} train examples")
print(f"Kept {len(test_dataset)} test examples")
if len(train_dataset) > 0:
    print("\nSample formatted example:")
    print(train_dataset[10]['text'])

Kept 1073 train examples
Kept 269 test examples

Sample formatted example:
<|im_start|>system
You are a natural disaster expert.<|im_end|>
<|im_start|>context
2 Min Read
At least one person was killed and 12 others injured after being hit by boulders following a landslide on the Gangotri National Highway near the Dabrani area in Uttarkashi district of Uttarakhand.
The incident took place at around noon on Friday. A woman labourer went missing, according to officials.
“The news of some people getting injured due to falling rocks near Dabrani on Gangotri National Highway is sad,” Uttarakhand Chief Minister Pushkar Singh Dhami wrote on social media site X.
He said that rescue and relief work is being carried out at a fast pace, while those injured have been rushed to the nearest health centre after giving them first aid.
“The concerned officials have been directed to provide proper treatment to the injured and to speed up relief and rescue operations,” the post read.
According to the offi

In [ ]:
def format_qwen_qa(example):
    """Handles multiple answers for Qwen fine-tuning"""
    try:
        # Extract context and question (common for all formats)
        context = example['context']
        question = example['question']

        # Initialize answer list
        answers = []

        # Case 1: Standard QA format with single answer
        if 'answer' in example:
            answers.append(str(example['answer']))

        # Case 2: SQUAD-like format with answer dictionary
        elif 'answers' in example:
            if isinstance(example['answers'], dict):
                ans_list = example['answers']['text'] if isinstance(example['answers']['text'], list) else [example['answers']['text']]
                answers.extend([str(a) for a in ans_list])
            else:
                answers.extend([str(a) for a in example['answers']])

        # Case 3: Nested QA format
        elif 'qas' in example:
            for qa in example['qas']:  # Handle multiple QAs per context
                if 'answers' in qa:
                    if isinstance(qa['answers'], dict):
                        ans_list = qa['answers']['text'] if isinstance(qa['answers']['text'], list) else [qa['answers']['text']]
                        answers.extend([str(a) for a in ans_list])
                    else:
                        answers.extend([str(a) for a in qa['answers']])

        # Format all answers into the Qwen prompt
        answers_str = "\n".join([f"- {ans}" for ans in answers]) if answers else "No answer available"

        return {
            "text": f"""<|im_start|>system
You are a natural disaster expert. Provide all possible answers.<|im_end|>
<|im_start|>context
{context}<|im_end|>
<|im_start|>question
{question}<|im_end|>
<|im_start|>answer
Possible answers:
{answers_str}<|im_end|>"""
        }

    except Exception as e:
        print(f"Skipping malformed example: {e}\nExample: {example}")
        return None


In [ ]:
# Check 5 random examples
for i in range(5):
    print(f"\nExample {i+1}:")
    print(train_dataset[i]['text'])
    print("-" * 50)


Example 1:
<|im_start|>system
You are a natural disaster expert.<|im_end|>
<|im_start|>context
Click here to go to economictimes.com Site will load in 10 seconds.Advertisement<|im_end|>
<|im_start|>question
What infrastructural damages are mentioned in 'Kanchenjunga Express mishap: Ashwini Vaishnaw meets injured at hospital; vows thorough investigation - The Economic Times'?<|im_end|>
<|im_start|>answer
The landslide blocked the road connecting Bhalukpong to Seppa<|im_end|>
--------------------------------------------------

Example 2:
<|im_start|>system
You are a natural disaster expert.<|im_end|>
<|im_start|>context
Machines were later moved to the scene to clear the road
Jammu: A massive landslide today blocked a road in Jammu and Kashmir's Kishtwar region as light to moderate rainfall hit isolated places in the region. The landslide blocked the Kishtwar-Paddar road in Kishtwar district, forcing the suspension of the vehicular movement.
A video of the landslide shows a large portio

In [ ]:
!pip install tiktoken

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen-1_8B",
    trust_remote_code=True,
    pad_token="<|endoftext|>"  # Qwen's default pad token
)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",  # Pad all sequences to same length
        truncation=True,       # Truncate to model's max length
        max_length=512,        # Adjust based on your context length
        return_tensors="pt"    # Return PyTorch tensors
    )

# Apply tokenization
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

print("Tokenized example keys:", list(tokenized_train[0].keys()))

Tokenized example keys: ['context', 'question', 'answers', 'text', 'input_ids', 'token_type_ids', 'attention_mask']


In [2]:
!pip uninstall -y torch torchvision torchaudio peft accelerate transformers -qqq
!pip cache purge
!rm -rf ~/.cache/huggingface
!rm -rf ~/.cache/torch

Files removed: 0


In [4]:
!pip install -q \
    torch==2.1.2 \
    torchvision==0.16.2 \
    torchaudio==2.1.2 \
    peft==0.7.1 \
    accelerate==0.24.1 \
    transformers==4.36.2 \
    bitsandbytes==0.41.3 \
    tiktoken==0.5.2 \
    sentencepiece==0.2.0

In [6]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj", "w1", "w2"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [9]:
#Load the base model with 4 bit quantization

#from transformers import AutoModelForCausalLM, BitsAndBytesConfig
#import torch
#
#bnb_config = BitsAndBytesConfig(
#    load_in_4bit=True,
#    bnb_4bit_quant_type="nf4",
#    bnb_4bit_compute_dtype=torch.bfloat16
#)
#
#model = AutoModelForCausalLM.from_pretrained(
#    "Qwen/Qwen-1_8B",
#    quantization_config=bnb_config,
#    device_map="auto",
#    trust_remote_code=True
#)


from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-1_8B",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True  # Critical for Qwen models
)

tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen-1_8B",
    trust_remote_code=True,
    pad_token="<|endoftext|>"
)

RuntimeError: Failed to import transformers.models.aria.configuration_aria because of the following error (look up to see its traceback):
No module named 'transformers.models.aria.configuration_aria'

In [10]:
from transformers import AutoModelForCausalLM

# Clear cache first
!rm -rf ~/.cache/huggingface/hub

# Then load with fresh download
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-1_8B",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    force_download=True,  # Force fresh download
    resume_download=False
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/910 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/910 [00:00<?, ?B/s]

configuration_qwen.py:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-1_8B:
- configuration_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


config.json:   0%|          | 0.00/910 [00:00<?, ?B/s]

RuntimeError: Failed to import transformers.models.aria.configuration_aria because of the following error (look up to see its traceback):
No module named 'transformers.models.aria.configuration_aria'

In [8]:
#apply lora to the model

from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()  # Should show ~0.5-1% trainable params

NameError: name 'model' is not defined